In [2]:
! pip install langchain
! pip install -U langchain-community tavily-python
! pip install --upgrade --quiet langchain-openai tavily-python
! pip install beautifulsoup4
! pip install faiss-cpu
! pip install langchainhub
! pip install -U wandb

  Using cached PyYAML-6.0.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (2.1 kB)
  Using cached aiohttp-3.9.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.4 kB)
  Using cached dataclasses_json-0.6.4-py3-none-any.whl.metadata (25 kB)
  Using cached jsonpatch-1.33-py2.py3-none-any.whl.metadata (3.0 kB)
  Using cached langchain_text_splitters-0.0.1-py3-none-any.whl.metadata (2.0 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.4 MB/s eta 0:00:00
  Using cached pydantic-2.6.4-py3-none-any.whl.metadata (85 kB)
  Using cached tenacity-8.2.3-py3-none-any.whl.metadata (1.0 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached attrs-23.2.0-py3-none-any.whl.metadata (9.5 kB)
  Using cached frozenlist-1.4.1-cp311-cp311-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached multidict-6.0.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86

In [3]:
import os
import yaml

config = yaml.safe_load(open("myconfig.yml"))

os.environ["OPENAI_API_KEY"] = config["OPENAI_API_KEY"]
os.environ["TAVILY_API_KEY"] = config["TAVILY_API_KEY"]
os.environ["LANGCHAIN_API_KEY"] = config["LANGCHAIN_API_KEY"]
os.environ["LANGCHAIN_HUB_API_KEY"] = config["LANGCHAIN_API_KEY"]
os.environ["LANGCHAIN_TRACING_V2"] = str(config["LANGCHAIN_TRACING_V2"]).lower()
os.environ["LANGCHAIN_ENDPOINT"] = config["LANGCHAIN_ENDPOINT"]
os.environ["LANGCHAIN_HUB_API_URL"] = config["LANGCHAIN_HUB_API_URL"]
os.environ["LANGCHAIN_WANDB_TRACING"] = str(config["LANGCHAIN_WANDB_TRACING"]).lower()
os.environ["WANDB_PROJECT"] = config["WANDB_PROJECT"]

# Planning

![images/Screenshot 2024-02-21 at 16.17.22.png](<images/Screenshot 2024-02-21 at 16.17.22.png>)

You might have come across various techniques aimed at improving the performance of large language models, such as offering tips or even jokingly threatening them. One popular technique is called "chain of thought," where the model is asked to think step by step, enabling self-correction. This approach has evolved into more advanced versions like "chain of thought with self-consistency" and the generalized "tree of thoughts," where multiple thoughts are created, re-evaluated, and consolidated to provide an output.

### Tree of Thoughts

The [@astropomeai tutorial](https://medium.com/@astropomeai/implementing-the-tree-of-thoughts-in-langchains-chain-f2ebc5864fac) on Tree of Thoughts is used as basis of this exercise but expanded with LLMOps tools.

In [4]:
from langchain.chains import LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI

In [5]:
from langchain import hub

cot_step1 = hub.pull("rachnogstyle/nlw_jan24_cot_step1")
cot_step2 = hub.pull("rachnogstyle/nlw_jan24_cot_step2")
cot_step3 = hub.pull("rachnogstyle/nlw_jan24_cot_step3")
cot_step4 = hub.pull("rachnogstyle/nlw_jan24_cot_step4")

In [6]:
model = "gpt-3.5-turbo"

chain1 = LLMChain(
    llm=ChatOpenAI(temperature=0, model=model),
    prompt=cot_step1,
    output_key="solutions"
)

chain2 = LLMChain(
    llm=ChatOpenAI(temperature=0, model=model),
    prompt=cot_step2,
    output_key="review"
)

chain3 = LLMChain(
    llm=ChatOpenAI(temperature=0, model=model),
    prompt=cot_step3,
    output_key="deepen_thought_process"
)

chain4 = LLMChain(
    llm=ChatOpenAI(temperature=0, model=model),
    prompt=cot_step4,
    output_key="ranked_solutions"
)

/home/p72admin/miniconda3/envs/autogpt/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [8]:
from langchain.chains import SequentialChain

overall_chain = SequentialChain(
    chains=[chain1, chain2, chain3, chain4],
    input_variables=["input", "perfect_factors"],
    output_variables=["ranked_solutions"],
    verbose=True
)

print(
        overall_chain(
            {
                "input": "human colonization of Mars",
                "perfect_factors": "The distance between Earth and Mars is very large, making regular resupply difficult"
            }
    )
)



> Entering new SequentialChain chain...

> Finished chain.
{'input': 'human colonization of Mars', 'perfect_factors': 'The distance between Earth and Mars is very large, making regular resupply difficult', 'ranked_solutions': "Developing sustainable habitats on Mars (Rank: 1)\n\nJustification: This solution has the highest potential for long-term success in establishing a sustainable human presence on Mars. By focusing on creating habitats that can support life in a harsh environment, this solution addresses the core challenge of colonization. The partnerships and strategies outlined provide a solid foundation for implementation, and the potential obstacles and unexpected outcomes are manageable with proper planning and flexibility.\n\nFinal thoughts: While this solution may require significant resources and effort, the potential benefits of establishing sustainable habitats on Mars are immense. By prioritizing this solution, we can work towards a future where humans can thrive on an

### ReAct prompt overview

Let's review [ReAct](https://python.langchain.com/docs/modules/agents/agent_types/react) prompt as it's defined in Langchain.

In [9]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react")
prompt.template

'Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {input}\nThought:{agent_scratchpad}'

### Self-ask with search

Let's review [self-ask with search](https://python.langchain.com/docs/modules/agents/agent_types/self_ask_with_search) as it's defined in Langchain.

In [10]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/self-ask-with-search")
prompt.template

'Question: Who lived longer, Muhammad Ali or Alan Turing?\nAre follow up questions needed here: Yes.\nFollow up: How old was Muhammad Ali when he died?\nIntermediate answer: Muhammad Ali was 74 years old when he died.\nFollow up: How old was Alan Turing when he died?\nIntermediate answer: Alan Turing was 41 years old when he died.\nSo the final answer is: Muhammad Ali\n\nQuestion: When was the founder of craigslist born?\nAre follow up questions needed here: Yes.\nFollow up: Who was the founder of craigslist?\nIntermediate answer: Craigslist was founded by Craig Newmark.\nFollow up: When was Craig Newmark born?\nIntermediate answer: Craig Newmark was born on December 6, 1952.\nSo the final answer is: December 6, 1952\n\nQuestion: Who was the maternal grandfather of George Washington?\nAre follow up questions needed here: Yes.\nFollow up: Who was the mother of George Washington?\nIntermediate answer: The mother of George Washington was Mary Ball Washington.\nFollow up: Who was the fathe

Approaches to read next:

**Reflexion** ([Shinn & Labash 2023](https://arxiv.org/abs/2303.11366)) is a framework to equips agents with dynamic memory and self-reflection capabilities to improve reasoning skills.

**Chain of Hindsight** (CoH; [Liu et al. 2023](https://arxiv.org/abs/2302.02676)) encourages the model to improve on its own outputs by explicitly presenting it with a sequence of past outputs, each annotated with feedback.